In [33]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def load_data(file_path):
    """Load and preprocess the baked food dataset."""
    df = pd.read_csv(file_path)
    df = df.dropna().reset_index(drop=True)  # Drop missing values
    
    # Create a combined description column
    df['CombinedDescription'] = df[['Food', 'ProductType', 'FlavorVariant']].astype(str).agg(' '.join, axis=1)
    return df

def preprocess_text(df, text_column):
    """Preprocess the text data for TF-IDF vectorization."""
    df[text_column] = df[text_column].astype(str).str.lower()
    return df

def compute_similarity(df, text_column, user_query, top_n=5):
    """Compute similarity scores and return top N recommendations."""
    vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = vectorizer.fit_transform(df[text_column])
    user_vector = vectorizer.transform([user_query])
    
    similarity_scores = cosine_similarity(user_vector, tfidf_matrix).flatten()
    top_indices = similarity_scores.argsort()[-top_n:][::-1]
    
    recommendations = df.iloc[top_indices].copy()
    recommendations['similarity_score'] = similarity_scores[top_indices]
    return recommendations

def main():
    """Main function to execute the recommendation system."""
    file_path = "C:/Users/Asus/BakedFoodNutritions.csv" 
    text_column = 'CombinedDescription'  
    
    df = load_data(file_path)
    df = preprocess_text(df, text_column)
    
    user_query = input("Enter your food preference: ")
    recommendations = compute_similarity(df, text_column, user_query)
    
    print("\nTop Recommendations:")
    print(recommendations[['Food', 'ProductType', 'FlavorVariant', 'similarity_score']])

if __name__ == "__main__":
    main()


Enter your food preference:  I want to eat something with high protein, high sodium



Top Recommendations:
                 Food                                   ProductType  \
55              Bread                                       Protein   
161          Crackers                                      Crackers   
424  Leavening Agents                              Leavening Agents   
655   Wonton Wrappers  Wonton Wrappers (Includes Egg Roll Wrappers)   
216           Glutino                                       Glutino   

                                    FlavorVariant  similarity_score  
55                                Includes Gluten          0.473796  
161                                    Low Sodium          0.424470  
424                                    Low-Sodium          0.241071  
655  Wonton Wrappers (Includes Egg Roll Wrappers)          0.000000  
216                                Milk Chocolate          0.000000  
